# Lake Depth RF to Raster Data

In [1]:
# System modules
import os
import time
from pprint import pprint

# Anything numeric related
import numpy as np
import matplotlib.pyplot as plt

# Anything geospatial related
import rasterio as rio

# Anything GPU/ML related
import cupy as cp
import cuml
import cudf
from models import custom_RF as crf

### Helper functions

In [2]:
def list_files(directory, extension):
    """ Helper function to read in files to list"""
    return list((f for f in os.listdir(directory) if f.endswith('.' + extension)))

In [3]:
def timer(start, end):
    """ Helper function for timing things """
    print (end - start)
    return (end - start)

### Useful file paths

In [4]:
TIF_PATH = '/att/nobackup/maronne/AVIRIS_2020/stacks/'
TIF_FILES = list(list_files(TIF_PATH, 'tif'))
CURRENT_TIF = TIF_FILES[0]
pprint(TIF_FILES)

['ang20170714t213741_corr_v2p9.tif',
 'ang20170709t230326_corr_v2p9.tif',
 'ang20170714t234307_corr_v2p9.tif',
 'ang20170709t224222_corr_v2p9.tif',
 'ang20170714t212855_corr_v2p9.tif',
 'ang20170715t005002_corr_v2p9.tif']


## Using rasterio for all raster-related things

The with statement is nice because it automatically closes the file after getting the necessary data

In [5]:
with rio.open(TIF_PATH+CURRENT_TIF) as raster_img:
    """ 
        The with statement is nice becuase it auto
        closes all rasterio tifs opened once below
        code is executed
    """
    n_cols = raster_img.width
    n_rows = raster_img.height
    n_bands = raster_img.count
    gt = raster_img.transform
    crs = raster_img.crs
    ndval = raster_img.nodata
    img_properties = (n_cols, n_rows, n_bands, gt, crs)
    
    #st_0 = time.time()
    # Create numpy array to mimic tif
    #img_nd = np.zeros((n_rows, n_cols, n_bands), np.float32) 
    img_nd = raster_img.read()
    #for b in range(n_bands):
    #    print(" - reading in band #", b)
    #    st_1 = time.time()
    #    img_nd[:, :, b] = raster_img.read(b+1) # Populate it with band pixel vals
    #    et_1 = time.time()
    #    timer(st_1, et_1)
    #et_0 = time.time()
    #timer(st_0, et_0)

In [6]:
print(img_nd.shape)
nshape = (img_nd.shape[1] * img_nd.shape[2], img_nd.shape[0])
print(nshape)

(425, 3116, 2896)
(9023936, 425)


In [7]:
from pprint import pprint
print('\n Image properties \n')
pprint(img_properties)
print("\n Image as ndarray \n")
pprint(img_nd)
print('\n Sample of bands, one row \n')
pprint(img_nd[:, 3000, 2000])
pprint(img_nd[:, 3000, 2000].shape)
# What one 'row' of bands look like
print(img_nd.shape)


 Image properties 

(2896,
 3116,
 425,
 Affine(5.200000000000116, 0.0, 581226.666764,
       0.0, -5.200000000000116, 7918447.266131187),
 CRS.from_epsg(32604))

 Image as ndarray 

array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999

In [8]:
img_nd_array = img_nd[:img_nd.shape[0], :, :].reshape(nshape)

In [9]:
print(img_nd_array.shape)
print(img_nd.shape)
pprint(img_nd[ :, 1500, 1000])
pprint(img_nd_array[(1500 * 1000), :])

(9023936, 425)
(425, 3116, 2896)
array([ 1.91083383e-02,  1.48145193e-02,  1.35489088e-02,  9.11048986e-03,
        1.26152514e-02,  1.43455742e-02,  1.51107376e-02,  1.87651869e-02,
        1.98740121e-02,  2.00550742e-02,  2.07723975e-02,  2.31170431e-02,
        1.97826661e-02,  2.28891391e-02,  2.43181586e-02,  2.37423871e-02,
        2.49232613e-02,  2.59839073e-02,  2.56036837e-02,  2.64618713e-02,
        2.62388233e-02,  2.66682040e-02,  2.81041656e-02,  2.70140655e-02,
        2.74790004e-02,  2.70548370e-02,  2.64715534e-02,  2.76322532e-02,
        2.88883187e-02,  2.82087214e-02,  2.86195129e-02,  2.96943504e-02,
        2.98443958e-02,  3.11550740e-02,  3.08688059e-02,  3.18734683e-02,
        3.18983495e-02,  3.10086645e-02,  3.18493657e-02,  2.94558797e-02,
        2.99837142e-02,  2.96758842e-02,  3.07513867e-02,  3.12435366e-02,
        2.99480055e-02,  2.96885949e-02,  2.92701516e-02,  2.95721833e-02,
        2.91617550e-02,  3.04874182e-02,  2.99751703e-02,  3.020821

### Convert our data to GPU-accelerated data
np array -> cupy array -> cudf Dataframe

In [11]:
import pandas as pd
cpu_img = pd.DataFrame(img_nd_array)


In [12]:
cdf_raster = cudf.from_pandas(cpu_img)

MemoryError: std::bad_alloc: CUDA error at: ../include/rmm/mr/device/cuda_memory_resource.hpp:68: cudaErrorMemoryAllocation out of memory

In [13]:
print(gpu_img.shape)
cdf_raster = cudf.DataFrame(gpu_img)

(6767952, 425)


MemoryError: std::bad_alloc: CUDA error at: ../include/rmm/mr/device/cuda_memory_resource.hpp:68: cudaErrorMemoryAllocation out of memory

In [ ]:
n_rows_raster, n_cols_raster = cdf_raster.shape
pprint(cdf_raster)
print(n_rows_raster)
print(n_cols_raster)

#### Load up our perviously trained random forest model

In [ ]:
model_load = crf.load_model('best_test_03.sav')

## Predictions from raster data
- We need to split it up due to GPU memory limitations

In [ ]:
index_0 = n_rows_raster//2
index_1 = index_0 * 2 
index_3 = n_rows_raster % 2 # Any indeces left out?
print(index_0)
print(index_1)

Predict first half then second half, no need for any third prediction since total is even

In [ ]:
predictions_1 = model_load.model.predict(cdf_raster[:index_0]) # Predict first half
predictions_2 = model_load.model.predict(cdf_raster[index_0:]) # Predict other hals
# No indeces left out so no need for any more predictions

In [ ]:
cp.unique(predictions_1) # Looks like this is ndvals

In [ ]:
cp.unique(predictions_2)

Put our predictions back by together by rows

In [ ]:
concat_predictions = cudf.concat([predictions_1, predictions_2], axis=0)
print(concat_predictions.shape)

In [ ]:
array_predictions = concat_predictions.to_array()

In [ ]:
array_predictions = array_predictions.reshape(img_nd[:, :, 0].shape).astype(np.float32)

In [ ]:
pprint(array_predictions)

In [ ]:
print(array_predictions.shape)

Make sure we put our ndvals back

In [ ]:
array_predictions[img_nd[:, :, 0] == -9999.0] = ndval

In [ ]:
pprint(array_predictions)

In [ ]:
file_name_no_extension = CURRENT_TIF.split('.', 1)[0]
print(file_name_no_extension)
file_name_predicted = file_name_no_extension + '_predicted_0.tif'
print(file_name_predicted)

In [ ]:
with rio.open(file_name_predicted, 
                   'w', 
                   driver='GTiff', 
                   height = array_predictions.shape[0],
                   width = array_predictions.shape[1],
                   count = 1,
                   dtype = array_predictions.dtype,
                   crs = crs,
                   transform = gt) as prediction_raster:
    prediction_raster.write(array_predictions, 1)

In [ ]:
with rio.open(file_name_predicted) as raster_img_predicted:
    """ 
        The with statement is nice becuase it auto
        closes all rasterio tifs opened once below
        code is executed
    """
    band1 = raster_img_predicted.read(1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 12))
plt.imshow(band1)

In [ ]:
with rio.open(TIF_PATH + CURRENT_TIF) as raster_img_original:
    """ 
        The with statement is nice becuase it auto
        closes all rasterio tifs opened once below
        code is executed
    """
    band1 = raster_img_original.read(1)
    band2 = raster_img_original.read(2)
    band3 = raster_img_original.read(3)

In [ ]:
rgb_original_tif = np.dstack((band1, band2, band3))
plt.figure(figsize=(12, 12))
plt.imshow(rgb_original_tif)